In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pylab as plt
import pandas as pd

In [12]:
class I:
    U = 0
    FL = 1
    SL = 2
    I = 3
    Tx = 4
    
    
class A:
    Inc = 0
    Mor = 1
    Covered = 2
    Eli = 3
    Yield = 4
    PCF = 5

    
    
class ModelTB:
    def get_y0(self):
        y0 = np.array([[970, 10, 10, 10, 0], np.zeros(5)]).T / 1000
        a0 = np.zeros(6)
        return np.concatenate([y0.reshape(-1), a0])
    
    def intervene(self, t, y, dy, da, acf):
        if acf['Cohort']:
            r_acf = - np.log(1 - acf['Coverage'])
            r_acf = min(r_acf, 20)
        else:
            r_acf = acf['Coverage']
        switch = acf['Switch']
        acf = r_acf * y[:, 0]
        eli = acf * np.array([1, 1, 1, 1, 0])
        
        da[A.Covered] += acf.sum()
        da[A.Eli] += eli.sum()
        da[A.Yield] += eli[I.I]
        
        if switch:
            dy[:, 0] -= eli
            dy[0:3, 1] += eli[0:3]
            dy[4, 1] += eli[3]
            dy[:, 0] += y[:, 1]
            dy[:, 1] -= y[:, 1]
        else:
            dy[3, 0] -= eli[3]
            dy[4, 1] += eli[3]
        
        return dy, da
    
    def __call__(self, t, ya, pars, acf=None):
        y, aux = ya[:-6], ya[-6:]
        y = y.reshape((5, 2))
        
        foi = pars['beta'] * y[I.I].sum()
        lat = pars['r_lat'] * y[I.FL]
        act = pars['r_act'] * y[I.FL]
        react = pars['r_react'] * y[I.SL]

        det = pars['r_det'] * y[I.I]
        txo = pars['r_tx'] * y[I.Tx]

        die_tb = pars['r_die_tb'] * y[I.I]
        r_die_bg = pars['r_die_bg'] - die_tb.sum() / y.sum()
        r_die_bg = max(r_die_bg, 0)
        die_bg = r_die_bg * y
        sc = pars['r_sc'] * y[I.I]

        dy = np.zeros_like(y)
        
        dy[I.U] -= foi * y[I.U]
        dy[I.FL] += foi * (y[I.U] + (1 - pars['p_im']) * y[I.SL]) - act - lat
        dy[I.SL] += lat - foi * (1 - pars['p_im']) * y[I.SL] + txo + sc - react
        dy[I.I] += act + react - det - sc
        dy[I.Tx] += det - txo

        dy -= die_bg
        dy[I.I] -= die_tb
        dy[I.U, 0] += die_bg.sum() + die_tb.sum()

        da = np.zeros_like(aux)
        da[A.Inc] += (act + react).sum()
        da[A.Mor] += die_tb.sum()
        
        if acf is not None:
            dy, da = self.intervene(t, y, dy, da, acf)
        
        return np.concatenate([dy.reshape(-1), da.reshape(-1)])
    
    def mea(self, t, ya):
        aux = ya[-6:]

        return {
            'Time': t,
            'Prev': ya[I.I],
            'Inc': aux[A.Inc],
            'Mor': aux[A.Mor],
            'Covered': aux[A.Covered],
            'Eli': aux[A.Eli],
            'Yield': aux[A.Yield],
            'PCF': aux[A.PCF]
        }

In [13]:
p = {
    'beta': 15,
    'r_act': 0.5 * 0.1 / 0.9,
    'r_lat': 0.5,
    'r_react': 0.002,
    'r_det': 1,
    'r_tx': 2,
    'r_die_tb': 0.1,
    'r_sc': 0.2,
    'r_die_bg': 0.05,
    'p_im': 0.6
}

In [14]:
model = ModelTB()
y0 = model.get_y0()

sol = solve_ivp(model, t_span = [0, 500], y0 = y0, args=(p, ))
ya0 = sol.y[:, -1]
ya0[-6:] = 0

In [15]:
def exp_discrete(model, ya0, coverage=0.1, interval=0.5):
    ts = np.linspace(0, 10, int(10 / interval) + 1)
    ya = ya0
    mss = list()
    mss.append(model.mea(0, ya))

    for t0, t1 in zip(ts[:-1], ts[1:]):
        sol = solve_ivp(model, t_span = [t0, t1], y0 = ya, args=(p, ))
        ya = sol.y[:, -1]

        ## ACF
        cov = coverage * interval
        y, aux = ya[:-6], ya[-6:]
        y = y.reshape((-1, 2))
        prop =  cov / y[:, 0].sum()
        prop = min(prop, 1)
        screened = y[:, 0] * prop
        eli = screened * np.array([1, 1, 1, 1, 0])
        found = eli[I.I]
        y[:, 0] -= eli
        y[0:3, 1] += eli[0:3]
        y[4, 1] += eli[3]
        aux[A.Covered] += screened.sum()
        aux[A.Eli] += eli.sum()
        aux[A.Yield] += eli[3]
        ya = np.concatenate([y.reshape(-1), aux])

        if round(t1) == t1:
            y, aux = ya[:-6], ya[-6:]
            y = y.reshape((-1, 2))
            y[:, 0] += y[:, 1]
            y[:, 1] = 0
            ya = np.concatenate([y.reshape(-1), aux])
            mss.append(model.mea(t1, ya))

    mss = pd.DataFrame(mss).assign(Coverage = coverage, Interval = interval)
    return mss

In [22]:
mss = list()

for cov in [0.1, 0.2, 0.5, 0.8]:
    for switch in [True, False]:
        for cohort in [True, False]:
            acf = {'Coverage': cov, 'Switch': switch, 'Cohort': cohort}
            
            ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, acf), dense_output=True)
            ms = pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)]).assign(**acf)
            mss.append(ms)
            
            
for c in [0.1, 0.2, 0.5, 1]:
    for i in [1, 0.5, 0.25, 0.1, 0.05, 0.1]:
        ms = exp_discrete(model, ya0, coverage=c, interval=i)
        mss.append(ms)

In [23]:
mss = pd.concat(mss)
mss.to_csv('out/coverage_test.csv')